In [26]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import sklearn

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

import xgboost as xg
from sklearn.multioutput import MultiOutputRegressor
from sklearn import metrics 

In [27]:
!pip install -U scikit-learn

In [28]:
train_df = pd.read_csv('train.csv')
display(train_df)

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2009,1,1,0,0,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0
1,2009,1,1,0,30,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0
2,2009,1,1,1,0,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0
3,2009,1,1,1,30,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0
4,2009,1,1,2,0,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175291,2018,12,31,21,30,51,555,168,4,19.4,20.8,1008,91.77,77.86,3.700,204.0,3.5,100
175292,2018,12,31,22,0,37,388,84,4,19.1,20.1,1008,93.88,83.03,3.800,209.0,3.2,100
175293,2018,12,31,22,30,15,115,18,7,19.1,19.6,1008,96.83,88.32,3.800,208.0,2.6,57
175294,2018,12,31,23,0,0,0,0,7,18.7,19.2,1009,96.84,94.34,3.700,206.0,2.1,0


In [29]:
test_df = pd.read_csv('test.csv')
display(test_df)

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,2019,1,1,0,0,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0,NaN,NaN,NaN
1,2019,1,1,0,30,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0,NaN,NaN,NaN
2,2019,1,1,1,0,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0,NaN,NaN,NaN
3,2019,1,1,1,30,3,18.2,18.3,1008,99.58,124.86,3.5,185,3.0,0,NaN,NaN,NaN
4,2019,1,1,2,0,0,18.0,18.0,1008,99.71,131.20,3.6,186,3.1,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019,12,31,21,30,0,5.6,11.1,1006,68.64,77.89,0.8,276,2.2,0,NaN,NaN,NaN
17516,2019,12,31,22,0,0,5.7,9.3,1006,78.31,83.06,0.8,274,1.7,0,NaN,NaN,NaN
17517,2019,12,31,22,30,0,5.7,8.5,1006,82.64,88.34,0.8,273,1.8,0,NaN,NaN,NaN
17518,2019,12,31,23,0,0,4.8,7.8,1006,81.49,94.37,0.8,272,1.8,0,NaN,NaN,NaN


In [30]:
train_df.loc[train_df['Fill Flag']>5, 'Fill Flag'] = 0
test_df.loc[test_df['Fill Flag']>5, 'Fill Flag'] = 0

In [31]:
train_df.describe()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
count,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000,175296.000000
mean,2013.500274,6.523549,15.727820,11.500000,15.000000,53.065426,330.065295,254.365490,2.540474,13.564431,18.266769,1003.185230,79.376366,89.729463,2.903732,188.877796,2.262871,0.173261
std,2.871908,3.448543,8.799349,6.922206,15.000043,64.201407,370.030821,327.705006,3.021879,8.196787,8.943096,5.935749,19.678235,40.085451,1.546979,99.014733,1.281902,0.789336
min,2009.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-17.000000,-9.000000,970.000000,7.190000,8.550000,0.122000,0.000000,0.000000,0.000000
25%,2011.000000,4.000000,8.000000,5.750000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,12.000000,1000.000000,65.590000,56.450000,1.534000,97.400000,1.300000,0.000000
50%,2013.500000,7.000000,16.000000,11.500000,15.000000,0.000000,0.000000,0.000000,1.000000,15.000000,20.000000,1000.000000,84.230000,89.860000,2.866000,200.000000,2.000000,0.000000
75%,2016.000000,10.000000,23.000000,17.250000,30.000000,103.000000,736.000000,535.000000,6.000000,21.000000,25.000000,1010.000000,97.950000,123.060000,4.223000,272.000000,3.000000,0.000000
max,2018.000000,12.000000,31.000000,23.000000,30.000000,565.000000,1040.000000,1063.000000,10.000000,26.900000,40.000000,1024.000000,100.000000,171.550000,7.700000,360.000000,12.900000,4.000000


In [32]:
train_df.dtypes

Year                    int64
Month                   int64
Day                     int64
Hour                    int64
Minute                  int64
Clearsky DHI            int64
Clearsky DNI            int64
Clearsky GHI            int64
Cloud Type              int64
Dew Point             float64
Temperature           float64
Pressure                int64
Relative Humidity     float64
Solar Zenith Angle    float64
Precipitable Water    float64
Wind Direction        float64
Wind Speed            float64
Fill Flag               int64
dtype: object

In [33]:
test_df.describe()

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Clearsky DHI,Clearsky DNI,Clearsky GHI
count,17520.0,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,17520.000000,0.0,0.0,0.0
mean,2019.0,6.526027,15.720548,11.500000,15.000000,2.715811,14.623550,19.929817,1006.751370,75.165104,89.701282,2.989498,184.470491,2.252871,0.033562,NaN,NaN,NaN
std,0.0,3.447950,8.796498,6.922384,15.000428,3.159985,7.273676,9.087679,4.742727,20.121986,40.089410,1.492317,97.672316,1.238956,0.364867,NaN,NaN,NaN
min,2019.0,1.000000,1.000000,0.000000,0.000000,0.000000,-8.700000,-4.800000,988.000000,16.870000,8.570000,0.300000,0.000000,0.100000,0.000000,NaN,NaN,NaN
25%,2019.0,4.000000,8.000000,5.750000,0.000000,0.000000,8.800000,13.000000,1003.000000,59.967500,56.432500,1.800000,92.000000,1.300000,0.000000,NaN,NaN,NaN
50%,2019.0,7.000000,16.000000,11.500000,15.000000,0.000000,16.600000,21.100000,1007.000000,79.235000,89.450000,3.000000,192.000000,2.000000,0.000000,NaN,NaN,NaN
75%,2019.0,10.000000,23.000000,17.250000,30.000000,7.000000,20.400000,26.400000,1010.000000,92.872500,123.040000,4.100000,265.000000,3.000000,0.000000,NaN,NaN,NaN
max,2019.0,12.000000,31.000000,23.000000,30.000000,9.000000,25.700000,38.700000,1023.000000,100.000000,171.540000,6.800000,360.000000,8.200000,4.000000,NaN,NaN,NaN


In [34]:
test_df.dtypes

Year                    int64
Month                   int64
Day                     int64
Hour                    int64
Minute                  int64
Cloud Type              int64
Dew Point             float64
Temperature           float64
Pressure                int64
Relative Humidity     float64
Solar Zenith Angle    float64
Precipitable Water    float64
Wind Direction          int64
Wind Speed            float64
Fill Flag               int64
Clearsky DHI          float64
Clearsky DNI          float64
Clearsky GHI          float64
dtype: object

In [35]:
CloudType = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15']
                     
FillFlag= ['0','1','2','3','4']

Month =['1','2','3','4','5','6','7','8','9','10','11','12']
Day =['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
Hour = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
Minute = ['0','30']

In [36]:
train_df1 = (train_df['Cloud Type']).astype(str)
x1 =pd.get_dummies(train_df1.astype(pd.CategoricalDtype(categories=CloudType)), prefix ='Cloud Type')
train_df1 = (train_df['Fill Flag']).astype(str)
x2 =pd.get_dummies(train_df1.astype(pd.CategoricalDtype(categories=FillFlag)), prefix ='Fill Flag')
train_df1 = (train_df['Month']).astype(str)
x3 =pd.get_dummies(train_df1.astype(pd.CategoricalDtype(categories=Month)), prefix ='Month')
train_df1 = (train_df['Day']).astype(str)
x4 =pd.get_dummies(train_df1.astype(pd.CategoricalDtype(categories=Day)), prefix ='Day')
train_df1 = (train_df['Hour']).astype(str)
x5 =pd.get_dummies(train_df1.astype(pd.CategoricalDtype(categories=Hour)), prefix ='Hour')
train_df1 = (train_df['Minute']).astype(str)
x6 =pd.get_dummies(train_df1.astype(pd.CategoricalDtype(categories=Minute)), prefix ='Minute')
x =pd.concat([x1,x2,x3,x4,x5,x6],axis=1)
x = x.astype('int64')


test_df1 = (test_df['Cloud Type']).astype(str)
xt1 =pd.get_dummies(test_df1.astype(pd.CategoricalDtype(categories=CloudType)), prefix ='Cloud Type')
test_df1 = (test_df['Fill Flag']).astype(str)
xt2 =pd.get_dummies(test_df1.astype(pd.CategoricalDtype(categories=FillFlag)), prefix ='Fill Flag')
test_df1 = (test_df['Month']).astype(str)
xt3 =pd.get_dummies(test_df1.astype(pd.CategoricalDtype(categories=Month)), prefix ='Month')
test_df1 = (test_df['Day']).astype(str)
xt4 =pd.get_dummies(test_df1.astype(pd.CategoricalDtype(categories=Day)), prefix ='Day')
test_df1 = (test_df['Hour']).astype(str)
xt5 =pd.get_dummies(test_df1.astype(pd.CategoricalDtype(categories=Hour)), prefix ='Hour')
test_df1 = (test_df['Minute']).astype(str)
xt6 =pd.get_dummies(test_df1.astype(pd.CategoricalDtype(categories=Minute)), prefix ='Minute')
x_test =pd.concat([xt1,xt2,xt3,xt4,xt5,xt6],axis=1)
x_test = x_test.astype('int64')


In [37]:
x_test.dtypes

Cloud Type_0    int64
Cloud Type_1    int64
Cloud Type_2    int64
Cloud Type_3    int64
Cloud Type_4    int64
                ...  
Hour_21         int64
Hour_22         int64
Hour_23         int64
Minute_0        int64
Minute_30       int64
Length: 90, dtype: object

In [38]:
display(x_test)

,Cloud Type_0,Cloud Type_1,Cloud Type_2,Cloud Type_3,Cloud Type_4,Cloud Type_5,Cloud Type_6,Cloud Type_7,Cloud Type_8,Cloud Type_9,...,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Minute_0,Minute_30
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
17516,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
17517,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
17518,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [39]:
min_max_scaler = MinMaxScaler()
non_cat_features = [x for x in train_df.columns if x not in ['Year','Month','Day','Hour','Minute','Cloud Type', 'Fill Flag','Clearsky DHI','Clearsky DNI','Clearsky GHI']]
train = train_df.copy()
test = test_df.copy()
train[non_cat_features] = min_max_scaler.fit_transform(train_df[non_cat_features])
test[non_cat_features] = min_max_scaler.fit_transform(test_df[non_cat_features])

In [40]:
X =pd.concat([x, train[non_cat_features]], axis=1)
X_test = pd.concat([x_test, test[non_cat_features]],axis=1)
y = train_df[['Clearsky DHI','Clearsky DNI','Clearsky GHI']]
y_test =test_df[['Clearsky DHI','Clearsky DNI','Clearsky GHI']]

In [41]:
display(X)

,Cloud Type_0,Cloud Type_1,Cloud Type_2,Cloud Type_3,Cloud Type_4,Cloud Type_5,Cloud Type_6,Cloud Type_7,Cloud Type_8,Cloud Type_9,...,Minute_0,Minute_30,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0.387244,0.285714,0.740741,0.734296,0.598773,0.049749,0.961389,0.240310
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0.410023,0.285714,0.740741,0.793233,0.636380,0.048562,0.961389,0.240310
2,0,0,0,0,1,0,0,0,0,0,...,1,0,0.387244,0.285714,0.740741,0.765866,0.674540,0.047506,0.966389,0.248062
3,0,0,0,0,1,0,0,0,0,0,...,0,1,0.387244,0.265306,0.740741,0.765866,0.713067,0.046978,0.966389,0.240310
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0.387244,0.265306,0.740741,0.746256,0.751963,0.046582,0.972222,0.232558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175291,0,0,0,0,1,0,0,0,0,0,...,0,1,0.829157,0.608163,0.703704,0.911324,0.425215,0.472156,0.566667,0.271318
175292,0,0,0,0,1,0,0,0,0,0,...,1,0,0.822323,0.593878,0.703704,0.934059,0.456933,0.485352,0.580556,0.248062
175293,0,0,0,0,0,0,0,1,0,0,...,0,1,0.822323,0.583673,0.703704,0.965844,0.489387,0.485352,0.577778,0.201550
175294,0,0,0,0,0,0,0,1,0,0,...,1,0,0.813212,0.575510,0.722222,0.965952,0.526319,0.472156,0.572222,0.162791


In [42]:
display(y)

,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
175291,51,555,168
175292,37,388,84
175293,15,115,18
175294,0,0,0


In [43]:
sklearn.__version__

'1.4.0'

In [44]:
def feature_select(n,X_train_stand,y_train):
    fs = SelectKBest(score_func=f_classif, k=n)
    # apply feature selection
    fs.fit(X_train_stand,y_train[['Clearsky DHI']])
    name1 =fs.get_feature_names_out()
    fs.fit(X_train_stand,y_train[['Clearsky DNI']])
    name2 =fs.get_feature_names_out()
    fs.fit(X_train_stand,y_train[['Clearsky GHI']])
    name3 =fs.get_feature_names_out()
    n1 = set(name1)
    n2 = set(name2)
    n3 = set(name3)
    name = n1.intersection(n2, n3)
    name = list(name)
    return name

In [45]:
Xtr, Xcv, ytr, ycv = train_test_split(X,y,test_size=0.35,random_state=27)

In [46]:
least_error = 9999999
feat_set =[]
#Decision Trees
for i in range(1, 128):
    features = feature_select(i, Xtr,ytr)
    clf1 = MultiOutputRegressor(xg.XGBRegressor(objective ='reg:squarederror',n_estimators = 10, seed = 123)).fit(Xtr[features],ytr)
    y_pred1 = clf1.predict(Xcv[features])
    y_pred1 = pd.DataFrame(y_pred1)
    a = metrics.mean_squared_error(ycv['Clearsky DHI'], y_pred1[0])
    b = metrics.mean_squared_error(ycv['Clearsky DNI'], y_pred1[1])
    c = metrics.mean_squared_error(ycv['Clearsky GHI'], y_pred1[2])
    print(i)
    error = (a+b+c)/3
    print('error', error)
    if error<least_error:
        least_error = error
        feat_set = features
        clf = clf1       
    
    print()
print(least_error)

1
error 2184.8778607701965

2
error 1871.4317522764022

3
error 1871.4317522764022

4
error 1060.0043375701473

5
error 1054.5894719519872

6
error 1054.5894719519872

7
error 1054.5894719519872

8
error 1054.5894719519872

9
error 1054.5894719519872

10
error 1045.558749138907

11
error 1037.8096038390972

12
error 1027.9996456505983

13
error 1009.1936835359638

14
error 1008.7468821843754

15
error 984.3939069149554

16
error 984.3939069149554

17
error 981.0861380436337

18
error 946.8570374136752

19
error 942.4675461305429

20
error 940.358624471255

21
error 920.8471121821435

22
error 922.288272405688

23
error 922.288272405688

24
error 922.288272405688

25
error 922.288272405688

26
error 922.288272405688

27
error 922.288272405688

28
error 922.288272405688

29
error 922.2882827219063

30
error 922.2882827219063

31
error 922.2882827219063

32
error 922.2882827219063

33
error 922.2882827219063

34
error 787.717583092423

35
error 787.717583092423

36
error 787.717583092423


In [55]:
clf.score(Xtr[feat_set],ytr)


0.9750176653926413

In [48]:
clf.score(Xcv[feat_set],ycv)

0.9737267345192643

In [49]:
y_t = clf.predict(X_test[feat_set])
y_test = pd.DataFrame(y_t)
y_test.describe()

,0,1,2
count,17520.000000,17520.000000,17520.000000
mean,54.505177,326.477783,253.401169
std,61.946651,352.514435,316.801880
min,1.495681,9.357579,7.196949
25%,1.495681,9.357579,7.196949
50%,4.520256,26.580645,9.561886
75%,109.437210,720.062408,525.908569
max,327.100983,957.661316,994.557251
